# Mathematical model
Here I describe the mathematical model used to estimate the mount axis.

## Celestial sphere and polar axis
We will describe positions on the celestial sphere using normalized 3-D Cartesian vectors

$$
\mathbf{x} = \begin{pmatrix}
x \\
y \\
z
\end{pmatrix},
$$

where $\|\mathbf{x}\| = 1$.
One way to parametrize the celestial sphere is using local alt-azimuthal coordinates.
Using the altitude $A$ and azimuth $\alpha$, we can write

$$
\mathbf{x} = \begin{pmatrix}
\cos(A)\cos(\alpha) \\
-\cos(A)\sin(\alpha) \\
\sin(A)
\end{pmatrix},
$$

where $A = \arcsin(z)$ and $\alpha = \arctan(-y/x)$.

In these coordinates, the $x$ axis points North, the $y$ axis points West, and the $z$ axis points to the
zenith. Azimuth $\alpha$ increases to the East.

The polar axis is given by $A_n = \phi_{obs}$, $\alpha_n = 0$, where $\phi_{obs}$ is the local 
latitude of the observer. In Cartesian coordinates, it is

$$
\mathbf{n} = \begin{pmatrix}
\cos(\phi_{obs}) \\
0 \\
\sin(\phi_{obs})
\end{pmatrix}.
$$

## Apparent motion of the stars
Due to the rotation of the Earth, a star at initial apparent position $\mathbf{x}_0$ is rotated on the
celestial sphere about the polar axis $\mathbf{n}$ with an angular velocity of $\Omega = 2\pi/T$,
where $T\approx 86164.0905\,s$ is the period of one sidereal day.

This motion is described by a rotation matrix 

$$
R(\mathbf{n}, \theta) = \mathbb{1} + \sin(\theta) K_{\mathbf{n}} +(1 - \cos(\theta)) K_{\mathbf{n}}^2.
$$

This is the Rodrigues rotation formula, where the antisymmetric matrix

$$
K_{\mathbf{n}} = \begin{pmatrix}
    0 & -n_z & n_y \\
    n_z & 0 & -n_x \\
    -n_y & n_x & 0
\end{pmatrix}
$$

is the infinitesimal generator of rotations about the axis $\mathbf{n}$.
NB: This formula can be rewritten as $R(\mathbf{n}, \theta) = \exp(\theta K_{\mathbf{n}})$ using a matrix
exponential.

Armed with this, we can finally write the apparent position of a star after a time $t$ as

$$
\mathbf{x}(t) = R(\mathbf{n}, -\Omega t) \mathbf{x}_0,
$$

where the minus sign comes from the fact that the Earth rotates counterclockwise as seen from the North celestial pole, and thus the celestial sphere rotates clockwise.

## Fitting the mount axis

Let's say we have an imperfectly polar-aligned mount that nonetheless rotates at $\Omega=2\pi/T$ (we are neglecting periodic error and similar issues).

If the mount axis is given by the normal vector $\mathbf{m}$, then initially pointing the camera it at a position
$\mathbf{x}_0$ on the celestial sphere, after a time $t$ the camera will point at

$$
\mathbf{x}(t) = R(\mathbf{m}, -\Omega t) \mathbf{x}_0.
$$

Note that the camera simply rotates about an axis different from the polar axis. A perfectly polar-aligned
mount has $\mathbf{m} = \mathbf{n}$.

In the astrotools code, we use plate-solving to obtain a set of measurements of camera pointing positions

$$
\mathbf{x}_1, \mathbf{x}_2, \mathbf{x}_3, \dots, \mathbf{x}_N
$$

and corresponding times $t_{12}, t_{23}, \dots, t_{N-1,N}$ such that $\mathbf{x}_{i+1} = R(\mathbf{m}, -\Omega
t_{i,i+1}) \mathbf{x}_i$.

We want to find the mount axis $\mathbf{m}$ to know how misaligned we are. To do this, we solve
the nonlinear least-squares problem which corresponds to minimizing the sum of squared errors

$$
E(\mathbf{m}) = \frac{1}{2} \sum_{i=1}^{N-1} \| \mathbf{x}_{i+1} - R(\mathbf{m}, -\Omega
t_{i,i+1})\mathbf{x}_i \|^2.
$$

The minimization is done numerically and the mount axis is parametrized by its altitude $A_m$ and azimuth
$\alpha_m$.

## Estimating drift velocity

Once the mount axis is known, we can estimate the velocity with which the camera's field drifts.
First we note that if we are looking at a position $\mathbf{x}(t) = R(\mathbf{n},-\Omega t)\mathbf{x}_0$ on the celestial
sphere, then the apparent motion in the frame of reference of the camera is given by

$$
\mathbf{x}_c(t) = R(\mathbf{m},\Omega t) R(\mathbf{n},-\Omega t)\mathbf{x}_{c,0}.
$$

We approximate the drift by looking at the angle between the camera positions after a time $t$,

$$
\cos\theta(t) = \mathbf{x}_c(t) \cdot \mathbf{x}_{c,0}.
$$

Taylor expanding for small $t$, we arrive at

$$
|\theta(t)| \approx v t, \qquad v = \Omega \|(\mathbf{m} - \mathbf{n})\times \mathbf{x}_{c,0}\|,
$$

where $v$ is the angular drift velocity.